# Create a table with SRA IDs, readstats QC data, and submitter metadata

**Below are the steps taken in this notebook:**
1. Installs, import Statements & Global Variable Definitions
2. Read in tables as dataframes
3. Merge dataframes
4. Write merged dataframe to data table and file


## Installs

In [2]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [3]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [4]:
from firecloud import fiss
import pandas as pd      
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [5]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

# table filenames (expected in the workspace bucket, see below)
submitter_metadata = '201112_UW_HPRC_PacBio_HiFi_Metadata_Submission_v0.2_kmmod.txt'
sra_metadata = 'metadata-8323615-processed-ok.tsv'

# submission id
submission_id = 'UW_HPRC_HiFi_Y1'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_UW_HPRC_HiFi_Y1
Workspace storage bucket: gs://fc-ed02327c-c4c0-4a7a-8339-50e7ebe38ce5/


# Read In tables as dataframes

## Readstats table
Find the individual readstats output files in the table and concatenate their contents.
Add the hifi filename and sample name.

In [6]:
# readstats
readstats_df = table_to_dataframe("readstats", workspace=WORKSPACE, workspace_namespace=PROJECT)
qc_list = []

for index, row in readstats_df.iterrows():

        sample_readstats_fp = row['ReadStatsReport']
        sample_readstats_fn = os.path.basename(sample_readstats_fp)
        df = pd.read_csv(sample_readstats_fp, header=None, sep='\t')
        df = df[df[0]=='sample.fastq']
        # add sample and file name
        #df = df.append({0:'undef', 1:'sample', 2:row['sample']}, ignore_index=True)
        df = df.append({0:'undef', 1:'filename', 2:os.path.basename(row['hifi'])}, ignore_index=True)
        # remove all but the variables (total_bp, quartile_25 etc)
        df2 = df.drop(df.columns[[0, 1]], axis=1)
        # make rownames ('total_bp', 'quartile_25' etc)
        df2.index = df.iloc[:,1]
        # remove the now meaningless 'sample.fastq' filename
        df2 = df2.drop(index=['file'])
        qc_list.append(df2)

# merge
readstats_df = pd.concat(qc_list, axis=1).transpose()
readstats_df.head()

1,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,quartile_50,quartile_75,N25,N50,N75,filename
2,1238949,22838743498,22.84,48,43615,18433,17496,18376,19340,17588,18478,19450,m54329U_201103_231616.ccs.bam
2,849857,17328670239,17.33,46,57023,20390,18712,19835,21255,18875,20054,21609,m64076_201013_225902.ccs.bam
2,1462110,29289007356,29.29,45,50257,20032,17344,19636,22397,17945,20385,23250,m64076_200311_082315.ccs.bam
2,1610516,30068543676,30.07,47,50034,18670,16676,18344,20318,17038,18766,20859,m64076_200201_051547.ccs.bam
2,1779397,32630636515,32.63,44,49139,18338,16407,18053,19983,16771,18467,20512,m64076_200203_181219.ccs.bam


## Submitter metadata table

In [7]:
meta_fp = os.path.join(bucket, submitter_metadata)
metadata_df = pd.read_csv(meta_fp, sep='\t')
metadata_df = metadata_df[['filename', 'sample_ID', 'instrument_model', 'shear_method', 'size_selection', 'ccs_algorithm', 
    'polymerase_version', 'seq_plate_chemistry_version', 'generator_facility', 'generator_contact']]
metadata_df.head()

,filename,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m54329U_200201_051510.ccs.bam,HG01123,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
1,m54329U_200203_181235.ccs.bam,HG01123,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
2,m54329U_200205_002609.ccs.bam,HG01123,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
3,m54329U_200206_215926.ccs.bam,HG01123,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
4,m64076_200306_185917.ccs.bam,HG01258,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu


## SRA table
Must download after finishing submission.
The downloaded table needs to be split in one row per file (instead of filename, filename2 <...> filenameN)

In [8]:
## Read in SRA file and split by filename

sra_file = os.path.join(bucket + sra_metadata)
sample_df = pd.read_csv(sra_file, sep='\t')

fnames = [x for x in sample_df.columns if x.startswith('filename')]
sra_df = pd.melt(sample_df, id_vars =['sample_name', 'accession'], value_vars = fnames, value_name='file')
sra_df= sra_df.dropna()
sra_df = sra_df[['file', 'accession', 'sample_name']]
sra_df.columns = ['filename', 'accession', 'sample']
sra_df.head()


,filename,accession,sample
0,m54329U_200201_051510.ccs.bam,SRR13684290,HG01123
1,m64076_200306_185917.ccs.bam,SRR13684289,HG01258
2,m64076_200201_051547.ccs.bam,SRR13684283,HG01358
3,m54329U_200306_185930.ccs.bam,SRR13684282,HG01361
4,m64076_200221_234426.ccs.bam,SRR13684281,HG01361


# Merge all dataframes

In [9]:
print(sra_df.shape, readstats_df.shape, metadata_df.shape)
# two samples of HG02572 were not uploaded to SRA with the rest of this batch

(46, 3) (47, 13) (46, 10)


In [10]:
set(readstats_df['filename']) - set(sra_df['filename'])
#m54329U_201103_231616.ccs.bam (HG002 sample) is not present in the submitter metadata or on SRA, but it is in the submissions 

{'m54329U_201103_231616.ccs.bam'}

In [11]:
merged_df = pd.merge(
    sra_df,
    readstats_df,
    on='filename')
merged_df = pd.merge(
    merged_df,
    metadata_df,
    on='filename')
merged_df.head()

,filename,accession,sample,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,...,N75,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m54329U_200201_051510.ccs.bam,SRR13684290,HG01123,2072143,34554070679,34.55,46,43126,16675,14984,...,18566,HG01123,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
1,m64076_200306_185917.ccs.bam,SRR13684289,HG01258,1353905,26041896578,26.04,44,49803,19234,16739,...,22588,HG01258,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
2,m64076_200201_051547.ccs.bam,SRR13684283,HG01358,1610516,30068543676,30.07,47,50034,18670,16676,...,20859,HG01358,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
3,m54329U_200306_185930.ccs.bam,SRR13684282,HG01361,1662447,33289500589,33.29,46,49882,20024,17940,...,22241,HG01361,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu
4,m64076_200221_234426.ccs.bam,SRR13684281,HG01361,352516,6760635369,6.76,45,42847,19178,18750,...,20718,HG01361,PacBio Sequel II,g-TUBE,SageELF,ccs 4.0.0 (commit SL-release-8.0.0),P2.0,C2.0,University of Washington,kmiyamot@uw.edu


# Upload To Tables

In [12]:
## Create table
#dataframe_to_table("post_sra_metadata", merged_df, WORKSPACE, PROJECT)
outf = os.path.join(bucket, submission_id + '_post_sra_metadata.tsv')
merged_df.to_csv(outf, sep="\t", index=False)

In [13]:
merged_df.shape

(46, 24)